In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

- Figure out how to deal with one card
- Figure out how to loop over cards on a page
- Figure out how to deal with multiple pages (categories)


In [3]:
response = requests.get('http://www.inshorts.com/en/read/entertainment')
soup = BeautifulSoup(response.text)

In [6]:
# .div is a little more specific
# select one will be more specific before we broaden the scope with just a select
card = soup.select_one('div.news-card')
output = {}
output['title'] = card.select_one('a.clickable').text.strip()
output

{'title': 'Sonu Sood asks for 50 liver transplants worth ₹12 crore as fees to promote a hospital'}

In [7]:
card

<div class="news-card z-depth-1" itemscope="" itemtype="http://schema.org/NewsArticle">
<span content="" itemid="https://inshorts.com/en/news/sonu-sood-asks-for-50-liver-transplants-worth-₹12-crore-as-fees-to-promote-a-hospital-1652178215731" itemprop="mainEntityOfPage" itemscope="" itemtype="https://schema.org/WebPage"></span>
<span itemprop="author" itemscope="itemscope" itemtype="https://schema.org/Person">
<span content="Daisy Mowke" itemprop="name"></span>
</span>
<span content="Sonu Sood asks for 50 liver transplants worth ₹12 crore as fees to promote a hospital" itemprop="description"></span>
<span itemprop="image" itemscope="" itemtype="https://schema.org/ImageObject">
<meta content="https://static.inshorts.com/inshorts/images/v1/variants/jpg/m/2022/05_may/10_tue/img_1652177483887_189.jpg?" itemprop="url"/>
<meta content="864" itemprop="width"/>
<meta content="483" itemprop="height"/>
</span>
<span itemprop="publisher" itemscope="itemscope" itemtype="https://schema.org/Organiza

In [10]:
# will give the whole thing of news card content
# multiple divs inside this, need to look at and specify
card_content = card.select_one('div.news-card-content')
output['content'] = card_content.select_one('div').text

In [12]:
# now looking at the author and the time that is listed
card_content

<div class="news-card-content news-right-box">
<div itemprop="articleBody">Sonu Sood, speaking to The Man magazine, recalled how he gave the money from his last few endorsements to charity. Citing an example, he said, "A gentleman...from Aster Hospitals connected with me...and said the group would like to collaborate with me...I said that I'll promote the hospitals, but give me 50 liver transplants. That's almost ₹12 crore in value."</div>
<div class="news-card-author-time news-card-author-time-in-content">
<a href="/prev/en/news/sonu-sood-asks-for-50-liver-transplants-worth-₹12-crore-as-fees-to-promote-a-hospital-1652178215731"><span class="short">short</span></a> by <span class="author">Daisy Mowke</span> / 
      <span class="time" content="2022-05-10T10:23:35.000Z" itemprop="dateModified">03:53 pm</span> on <span class="date">10 May</span>
</div>
</div>

In [13]:
card_content.select_one('.author').text

'Daisy Mowke'

In [14]:
output['author'] = card_content.select_one('.author').text

In [15]:
card_content

<div class="news-card-content news-right-box">
<div itemprop="articleBody">Sonu Sood, speaking to The Man magazine, recalled how he gave the money from his last few endorsements to charity. Citing an example, he said, "A gentleman...from Aster Hospitals connected with me...and said the group would like to collaborate with me...I said that I'll promote the hospitals, but give me 50 liver transplants. That's almost ₹12 crore in value."</div>
<div class="news-card-author-time news-card-author-time-in-content">
<a href="/prev/en/news/sonu-sood-asks-for-50-liver-transplants-worth-₹12-crore-as-fees-to-promote-a-hospital-1652178215731"><span class="short">short</span></a> by <span class="author">Daisy Mowke</span> / 
      <span class="time" content="2022-05-10T10:23:35.000Z" itemprop="dateModified">03:53 pm</span> on <span class="date">10 May</span>
</div>
</div>

In [18]:
#.attrs will give a dictionary of elements
card_content.select_one('.time').attrs['content']

'2022-05-10T10:23:35.000Z'

In [58]:
#function takes in a card and in order to produce a dictionary we use the return
# category arguemnt is added so we can have the category returned later.
def parse_news_card(card, category):
    card = soup.select_one('div.news-card')
    output = {}
    output['category'] = category
    output['title'] = card.select_one('a.clickable').text.strip()
    
    card_content = card.select_one('div.news-card-content')
    output['content'] = card_content.select_one('div').text
    output['author'] = card_content.select_one('.author').text
    output['published'] = card_content.select_one('.time').attrs['content']
    
    return output

In [59]:
#testing function to see if it works on one card
card = soup.select_one('.news-card')
parse_news_card(card)

TypeError: parse_news_card() missing 1 required positional argument: 'category'

In [28]:
articles = [parse_news_card(card) for card in soup.select('.news-card')]

In [23]:
# step above was added afterwards
# need function to work on multiple cards
for card in soup.select('.news-card'):
    print(parse_news_card(card))

{'title': 'Sonu Sood asks for 50 liver transplants worth ₹12 crore as fees to promote a hospital', 'content': 'Sonu Sood, speaking to The Man magazine, recalled how he gave the money from his last few endorsements to charity. Citing an example, he said, "A gentleman...from Aster Hospitals connected with me...and said the group would like to collaborate with me...I said that I\'ll promote the hospitals, but give me 50 liver transplants. That\'s almost ₹12 crore in value."', 'author': 'Daisy Mowke', 'published': '2022-05-10T10:23:35.000Z'}
{'title': 'Sonu Sood asks for 50 liver transplants worth ₹12 crore as fees to promote a hospital', 'content': 'Sonu Sood, speaking to The Man magazine, recalled how he gave the money from his last few endorsements to charity. Citing an example, he said, "A gentleman...from Aster Hospitals connected with me...and said the group would like to collaborate with me...I said that I\'ll promote the hospitals, but give me 50 liver transplants. That\'s almost ₹

In [24]:
# function is returning multiple dictionaries which is good
# however this is not useful to just have print statements, list comprehension is needed


In [25]:
# with a for loop it is a candidate for list comprehension

In [44]:
# putting it together
def parse_entertainment_page():
    response = requests.get('http://www.inshorts.com/en/read/entertainment')
    soup = BeautifulSoup(response.text)
    articles = [parse_news_card(card) for card in soup.select('.news-card')]
    return articles


In [57]:
# this function will now parse the entertainment page and return dictionaries
# two leveles deep. this function calls another function 'parse_news_card'
# one grabs all the news cards, the other function converts cards into dictionaries
parse_entertainment_page()

[{'title': "Ranveer's 'Jayeshbhai...' cleared for release, court orders new disclaimers to be added",
  'content': "Delhi High Court has cleared Ranveer Singh-starrer 'Jayeshbhai Jordaar' for theatrical release on May 13, asking the producers to add new disclaimers. It was hearing a petition that sought removal of a scene involving pre-natal sex determination test. Earlier, the court had demanded a screening, asking producers to show them the relevant scene or face a stay on the release.",
  'author': 'Daisy Mowke',
  'published': '2022-05-10T12:05:33.000Z'},
 {'title': "Ranveer's 'Jayeshbhai...' cleared for release, court orders new disclaimers to be added",
  'content': "Delhi High Court has cleared Ranveer Singh-starrer 'Jayeshbhai Jordaar' for theatrical release on May 13, asking the producers to add new disclaimers. It was hearing a petition that sought removal of a scene involving pre-natal sex determination test. Earlier, the court had demanded a screening, asking producers to s

### need a function that parses more than just the entertainment page and any category
http://www.inshorts.com/en/read/business

- only difference is the last word in each url, just make it generic

In [60]:
# category to list[dict]
# takes in a category and returns a list of dictionaries
# every dictionary in list of dictionaries, use the .extend fucntion. takes in a list and adds every eleement from the lsit
# .append: takes in 1 element and adds it to the list
def parse_inshorts_page(category):
    url = 'http://www.inshorts.com/en/read/' + category
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    articles = [parse_news_card(card, category) for card in soup.select('.news-card')]
    return articles

In [61]:
categories = ['business', 'sports', 'entertainment', 'technology']
# main overall list of articles from every category
articles = []

for category in categories:
    category_articles = parse_inshorts_page(category)
    articles.extend(category_articles)

In [62]:
#checking to make sure the function works! it does
articles[0]

{'category': 'business',
 'title': "Ranveer's 'Jayeshbhai...' cleared for release, court orders new disclaimers to be added",
 'content': "Delhi High Court has cleared Ranveer Singh-starrer 'Jayeshbhai Jordaar' for theatrical release on May 13, asking the producers to add new disclaimers. It was hearing a petition that sought removal of a scene involving pre-natal sex determination test. Earlier, the court had demanded a screening, asking producers to show them the relevant scene or face a stay on the release.",
 'author': 'Daisy Mowke',
 'published': '2022-05-10T12:05:33.000Z'}

In [63]:
# lets make it all a dataframe now
# missing a piece of information though, which category each article belongs to
# each row is one dictionary, dictionary doesnt have a category key so it needs to be added
# going back to the parse_news_function and inshorts modifying
# added it in function definition, output dictionary, and inshorts article
pd.DataFrame(articles)


,category,title,content,author,published
0,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
1,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
2,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
3,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
4,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
...,...,...,...,...,...
95,technology,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
96,technology,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
97,technology,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
98,technology,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z


## turn this all into one function

In [64]:
def get_news_articles():
    categories = ['business', 'sports', 'entertainment', 'technology']
# main overall list of articles from every category
    articles = []

    for category in categories:
        category_articles = parse_inshorts_page(category)
        articles.extend(category_articles)
    
    return pd.DataFrame(articles)

In [65]:
df = get_news_articles()
df.head()

,category,title,content,author,published
0,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
1,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
2,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
3,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
4,business,"Ranveer's 'Jayeshbhai...' cleared for release,...",Delhi High Court has cleared Ranveer Singh-sta...,Daisy Mowke,2022-05-10T12:05:33.000Z
